In [1]:
#get the lat lon of HD/Lowes HQ? maybe hardcode?
#LOW Mooresville, NC 35.5859° N, 80.8081° W
#HD Altanta GA 33.7490° N, 84.3880° W
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import geopy.distance

In [2]:
entry_data = pd.read_csv("entry.csv",sep='\t')
states = pd.read_csv('states.csv')
states = states.rename(columns={'STATE':'state'})
entry_data = entry_data.merge(states,on='state')

In [3]:
#set up geolocator
geolocator = Nominatim(user_agent="user",timeout=10)
low_loc = geolocator.geocode('Mooresville, NC', exactly_one=False)[0][1]
hd_loc = geolocator.geocode('Atlanta, GA', exactly_one=False)[0][1]

In [4]:
def lat_lon_dist( coord1 , coord2 ):
    lat1,lon1 = coord1
    lat2,lon2 = coord2
    R = 6373.0

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

entry_data_loc = entry_data.copy()
entry_data_loc['lat'] = np.nan
entry_data_loc['lon'] = np.nan
entry_data_loc['low_dist'] = np.nan
entry_data_loc['hd_dist'] = np.nan


for index, row in entry_data_loc.iterrows():
    location_results = geolocator.geocode(row['city']+', '+row['STUSAB'], exactly_one=False)
    if location_results is not None:
        row['lat'] = location_results[0][1][0]
        row['lon'] = location_results[0][1][1]
        row['low_dist'] = geopy.distance.geodesic(location_results[0][1], low_loc).km
        row['hd_dist'] = geopy.distance.geodesic(location_results[0][1], hd_loc).km
    entry_data_loc.loc[index] = row
    
entry_data_loc.to_csv('entry_loc.csv')